# Operation Room Scheduling Optimization With Pyomo

### Pyomo Solver - Concrete Model

Pyomo is a powerful optimization modeling language for mathematical programming in Python. It provides a convenient way to formulate optimization problems and interface with various solvers to find optimal solutions.

Pyomo's concrete model() is a feature that allows you to define and solve optimization problems using a declarative approach. It provides a high-level, intuitive representation of the optimization problem, making it easier to formulate and solve complex scheduling problems.

The elements of a concrete model() include:

-  **Decision Variables**: Decision variables are variables that you define in an optimization problem to represent the quantities or choices that you want to determine. These variables determine the values that will be optimized. They capture the unknowns or decision points in the problem. 

-  **Objective Function**: The objective function is a mathematical expression that defines the quantity you want to maximize or minimize in an optimization problem. It represents the goal or measure of performance that you aim to optimize. The objective function typically depends on the decision variables.

-  **Constraints**: Constraints are conditions or limitations that must be satisfied in an optimization problem. They represent the requirements or restrictions on the values of the decision variables. Constraints define the feasible region of the problem and help to narrow down the set of possible solutions. 

#### Data & Context - Operating Room Scheduling

##### Define the problem

The problem involves optimizing the scheduling of operating rooms in a healthcare facility over a three-month period. The objective is to maximize the number of procedures that can be scheduled during this time. The model aims to allocate procedures to available operating rooms while considering their durations and any constraints such as room availability and procedure compatibility. 

By finding an optimal solution, the goal is to improve resource utilization and increase the number of scheduled procedures, ultimately enhancing operational efficiency.

##### Decision Variables

* Schedule: A scheduling decision variable that indicates if a procedure is scheduled or not. It is a binary variable, with a value of 1 if the procedure is scheduled, and 0 if it is not scheduled. It is indexed by the Encounter ID, OR Suite, Day, and Time.

##### Parameters

* OR_Suites: Operating Room Suites - A total of 3 OR Suites
* CPT_Codes: CPT Codes - A total of 32 CPT Codes
* Booked_Time: Booked Time - The booked time estimated for each procedure (unique for each CPT Code)
* Encounter ID: Encounter ID - A unique identifier for each procedure, with a total of 2172 procedures requested. Only 15% of them will go into the schedule excercise, through a random assigment, and with the purpose to make the calculations more managable.

##### Constraints

In order to provide the optimization nature to the problem, the following constraints were added:

* The total booked time in a day cannot surpass 24 hours (3 OR Suites * 8 hours).
* Each OR can only perform one procedure at a time.
* Each procedure can only be performed in one OR Suite.
* Each procedure can only be scheduled once.
* The total booked time for each OR Suite cannot surpass 56 hours weekly.
* The total booked time for each OR and Day cannot surpass 8 hours.

##### Data

The data consist of 2172 records to attemp to schedule. The data was created by somebody that specializes in Data Analytics for Health Care Providers, and comes from a public source: https://data.world/drjfalk/operating-room-utilization

In [105]:
# Import the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from pyomo.environ import *

In [106]:
# Read the Excel data 
df = pd.read_excel('2022_Q1_OR_Utilization.xlsx')

In [107]:
# Explore the data
df.head()

,Encounter ID,Date,OR Suite,Service,CPT Code,CPT Description,Booked Time (min),OR Schedule,Wheels In,Start Time,End Time,Wheels Out
0,10001,2022-01-03,1,Podiatry,28110,"Partial ostectomy, fifth metatarsal head",90,2022-01-03 07:00:00,2022-01-03 07:05:00,2022-01-03 07:32:00,2022-01-03 09:05:00,2022-01-03 09:17:00
1,10002,2022-01-03,1,Podiatry,28055,"Neurectomy, intrinsic musculature of foot",60,2022-01-03 08:45:00,2022-01-03 09:48:00,2022-01-03 10:13:00,2022-01-03 11:01:00,2022-01-03 11:12:00
2,10003,2022-01-03,1,Podiatry,28297,Lapidus bunionectomy,150,2022-01-03 10:00:00,2022-01-03 11:50:00,2022-01-03 12:20:00,2022-01-03 12:42:00,2022-01-03 12:58:00
3,10004,2022-01-03,1,Podiatry,28296,Bunionectomy with distal osteotomy,120,2022-01-03 12:45:00,2022-01-03 13:29:00,2022-01-03 13:53:00,2022-01-03 14:50:00,2022-01-03 15:02:00
4,10005,2022-01-03,2,Orthopedics,27445,"Arthroplasty, knee, hinge prothesis",120,2022-01-03 07:00:00,2022-01-03 07:15:00,2022-01-03 07:50:00,2022-01-03 09:38:00,2022-01-03 09:51:00


In [108]:
# Describe the data
df.describe()

,Encounter ID,OR Suite,CPT Code,Booked Time (min)
count,2172.000000,2172.000000,2172.000000,2172.000000
mean,11086.500000,4.288674,44881.405617,77.189227
std,627.146713,2.163514,18087.419079,30.430015
min,10001.000000,1.000000,14060.000000,30.000000
25%,10543.750000,3.000000,28296.000000,60.000000
50%,11086.500000,4.000000,42826.000000,60.000000
75%,11629.250000,6.000000,66982.000000,90.000000
max,12172.000000,8.000000,69436.000000,180.000000


In [109]:
df.shape

(2172, 12)

In [110]:
# Count of unique Encounter ID
df['Encounter ID'].nunique()

2172

In [111]:
# Count of unique CPT codes
df['CPT Code'].nunique()
print(df['CPT Code'].nunique())

32


In [112]:
# Does each procedure has the same booked time?
print(df.groupby('CPT Code')['Booked Time (min)'].nunique())

CPT Code
14060    1
15773    1
17110    1
26045    1
26356    1
26735    1
27130    1
27445    1
28055    1
28060    1
28110    1
28285    1
28289    1
28296    1
28297    1
28820    1
29877    1
30400    1
30520    1
36901    1
42826    1
43775    1
47562    1
52353    1
55250    1
55873    1
57460    1
58562    1
64721    1
66982    2
69421    1
69436    1
Name: Booked Time (min), dtype: int64


For simplicity purposes the procedures times have been standarized for individual codes.

In [113]:
# Checking for CPT Code 66982 booked time
print(df[df['CPT Code'] == 66982]['Booked Time (min)'].unique())

[45 30]


In [114]:
# Transforming the values from 30 to 45
df.loc[df['CPT Code'] == 66982, 'Booked Time (min)'] = 45

In [115]:
# Checking for CPT Code 66982 booked time
print(df[df['CPT Code'] == 66982]['Booked Time (min)'].unique())

[45]


In [116]:
# Does each procedure has the same booked time?
print(df.groupby('CPT Code')['Booked Time (min)'].nunique())

CPT Code
14060    1
15773    1
17110    1
26045    1
26356    1
26735    1
27130    1
27445    1
28055    1
28060    1
28110    1
28285    1
28289    1
28296    1
28297    1
28820    1
29877    1
30400    1
30520    1
36901    1
42826    1
43775    1
47562    1
52353    1
55250    1
55873    1
57460    1
58562    1
64721    1
66982    1
69421    1
69436    1
Name: Booked Time (min), dtype: int64


In [117]:
# Dropping the not needed columns from the df dataframe
df.drop(['Date', 'OR Suite', 'OR Schedule',	'Wheels In', 'Start Time', 'End Time', 'Wheels Out', 'Service'], axis=1, inplace=True)

In [118]:
# Transform the Booked Time (min) column to hours
df['Booked Time (min)'] = df['Booked Time (min)'] / 60

In [119]:
# Change the Booked Time (min) column name to Booked Time (hr)
df.rename(columns={'Booked Time (min)': 'Booked Time (hr)'}, inplace=True)

### Data Reduction

Considering the computing effort, and that the total amount of records represent the execution of 90 days of operations, for this excercise I will take the proportion of 2 weeks, which is around 15% of the data. This will allow to run the model in a reasonable amount of time.

In [120]:
# Randoming select 15% of the df data
df = df.sample(frac=0.15, random_state=1)

In [121]:
df.head()

,Encounter ID,CPT Code,CPT Description,Booked Time (hr)
952,10953,42826,Tonsillectomy,1.0
1285,11286,28110,"Partial ostectomy, fifth metatarsal head",1.5
726,10727,28285,"Correction, hammertoe",1.0
2016,12017,55250,Vasectomy,1.0
406,10407,69436,"Tympanostomy, general anesthesia",1.0


In [122]:
df.shape

(326, 4)

In [123]:
# Add all DF Booked Time (hr) 
df['Booked Time (hr)'].sum()

413.25

The total hours available for 1 week of 3 Operating Rooms is 168 hours, which is 8 hours a day, 7 days a week, for the 3 OR Suites. The total "Demand" hours coming from the data frame is 413 hours, which is 2.45 times the total available hours. 

In [124]:
# Create a optimization model to schedule the Operating Rooms for a period of 3 months

# The model will be based on the following assumptions:

# 1. The ORs will be scheduled for 1 week.
# 2. The ORs will be scheduled for 7 days a week, teams will performed during the week and weekends
# 3. The ORs will be scheduled for 8 hours a day, and the health care team will be working from 8am to 5pm

# Create the model
model = ConcreteModel()

# Create the sets
model.CPT = Set(initialize = df['CPT Code'].unique())
model.Encounter = Set(initialize = df['Encounter ID'].unique())
model.daynumber = Set(initialize = range(1, 8))
model.hour = Set(initialize = range(8, 17))
model.OR = Set(initialize = range(1, 4))

# Create the parameters
model.BookedTime = Param(model.CPT, initialize = df.set_index('CPT Code')['Booked Time (hr)'].to_dict())
model.EncounterCPT = Param(model.Encounter, initialize = df.set_index('Encounter ID')['CPT Code'].to_dict())
model.ORDayCapacity = Param(model.OR, initialize = {1: 8, 2: 8, 3: 8})


# Create the variables
model.x = Var(model.Encounter, model.OR, model.daynumber, model.hour, domain = Binary)

# Create the objective function
def obj_rule(model):
    return sum(model.x[e, o, d, h] * model.BookedTime[model.EncounterCPT[e]] for e in model.Encounter for o in model.OR for d in model.daynumber for h in model.hour)

# Create the objective 
model.obj = Objective(rule = obj_rule, sense = maximize)

# Create the constraints

# the total sum of BookedTime per day cannot be greater than 24 hours (8 hours per day * 3 PR)
def constraint1_rule(model, d):
    return sum(model.x[e, o, d, h] * model.BookedTime[model.EncounterCPT[e]] for e in model.Encounter for o in model.OR for h in model.hour) <= 24

# Each OR can only perform one surgical procedure at a time
def constraint2_rule(model, o, d, h):
    return sum(model.x[e, o, d, h] for e in model.Encounter) <= 1

# Each procedure can only be performed in one OR
def constraint3_rule(model, e, d, h):
    return sum(model.x[e, o, d, h] for o in model.OR) <= 1

# model.encounter (procedure ID) will only by assigned one time
def constraint4_rule(model, e, o, d, h):
    return sum(model.x[e, o, d, h] for o in model.OR for d in model.daynumber for h in model.hour) <= 1

# the total sum of BookedTime per OR cannot be greater than 56 hours (8 hours per day * 7 days)
def constraint5_rule(model, o):
    return sum(model.x[e, o, d, h] * model.BookedTime[model.EncounterCPT[e]] for e in model.Encounter for d in model.daynumber for h in model.hour) <= 56

# the total sum of BookedTime per OR and day cannot be greater than 8 hours (8 hours per day)
def constraint6_rule(model, o, d):
    return sum(model.x[e, o, d, h] * model.BookedTime[model.EncounterCPT[e]] for e in model.Encounter for h in model.hour) <= 8


# Create the constraints
model.constraint1 = Constraint(model.daynumber, rule = constraint1_rule)
model.constraint2 = Constraint(model.OR, model.daynumber, model.hour, rule = constraint2_rule)
model.constraint3 = Constraint(model.Encounter, model.daynumber, model.hour, rule = constraint3_rule)
model.constraint4 = Constraint(model.Encounter, model.OR, model.daynumber, model.hour, rule = constraint4_rule)
model.constraint5 = Constraint(model.OR, rule = constraint5_rule)
model.constraint6 = Constraint(model.OR, model.daynumber, rule = constraint6_rule)

In [125]:
# Solve the model
solver = SolverFactory('glpk', executable='c:/glpk-4.65/w64/glpsol')
solver.solve(model);

In [126]:
# Print the results and save them
results = pd.DataFrame(columns = ['Encounter ID', 'OR', 'Day', 'Hour'])

for e in model.Encounter:
    for o in model.OR:
        for d in model.daynumber:
            for h in model.hour:
                if model.x[e, o, d, h]() == 1:
                    results = results.append({'Encounter ID': e, 'OR': o, 'Day': d, 'Hour': h}, ignore_index = True);

C:\Users\dario\AppData\Local\Temp\ipykernel_38180\1742789504.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Encounter ID': e, 'OR': o, 'Day': d, 'Hour': h}, ignore_index = True);
C:\Users\dario\AppData\Local\Temp\ipykernel_38180\1742789504.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Encounter ID': e, 'OR': o, 'Day': d, 'Hour': h}, ignore_index = True);
C:\Users\dario\AppData\Local\Temp\ipykernel_38180\1742789504.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Encounter ID': e, 'OR': o, 'Day': d, 'Hour': h}, ignore_index = True);
C:\Users\dario\AppData\Local\Temp\ipykernel_38180\1742789504.py:9: FutureWarning: The frame.append m

In [127]:
results

,Encounter ID,OR,Day,Hour
0,10407,1,3,9
1,11734,2,7,11
2,10135,2,3,8
3,11267,2,3,10
4,10620,3,7,10
...,...,...,...,...
79,11970,3,3,10
80,11756,3,2,8
81,11732,2,7,10
82,11816,3,2,10


In [128]:
# Cross the results Encounter ID with DF Booking Time (hr)
results = pd.merge(results, df[['Encounter ID', 'Booked Time (hr)']], on = 'Encounter ID', how = 'left')

In [129]:
results.head()

,Encounter ID,OR,Day,Hour,Booked Time (hr)
0,10407,1,3,9,1.0
1,11734,2,7,11,2.0
2,10135,2,3,8,2.0
3,11267,2,3,10,2.0
4,10620,3,7,10,1.5


In [130]:
# Calculare total Booked Time (hr) per OR
results.groupby('OR')['Booked Time (hr)'].sum()

OR
1    56.0
2    56.0
3    56.0
Name: Booked Time (hr), dtype: float64

In [131]:
# Calculare total Booked Time (hr) per Day
results.groupby('Day')['Booked Time (hr)'].sum()

Day
1    24.0
2    24.0
3    24.0
4    24.0
5    24.0
6    24.0
7    24.0
Name: Booked Time (hr), dtype: float64

In [132]:
# Calculare total Booked Time (hr) per OR and Day
results.groupby(['OR', 'Day'])['Booked Time (hr)'].sum()

OR  Day
1   1      8.0
    2      8.0
    3      8.0
    4      8.0
    5      8.0
    6      8.0
    7      8.0
2   1      8.0
    2      8.0
    3      8.0
    4      8.0
    5      8.0
    6      8.0
    7      8.0
3   1      8.0
    2      8.0
    3      8.0
    4      8.0
    5      8.0
    6      8.0
    7      8.0
Name: Booked Time (hr), dtype: float64

In [144]:
# Calculate the End Time of each scheduled procedure
results['End Time'] = results['Hour'] + results['Booked Time (hr)']

In [169]:
# Generate a Table that Orders the Encounter ID by OR, Day and Hour
results = results.sort_values(by = ['OR', 'Day', 'Hour'])


In [171]:
# Present the Schedule for Day 1 and OR 1 in a table
results[(results['OR'] == 1) & (results['Day'] == 1)]

,Encounter ID,OR,Day,Hour,Booked Time (hr),End Time
62,10817,1,1,08:00:00,3.0,11.0
15,10481,1,1,09:00:00,2.0,11.0
37,10127,1,1,10:00:00,3.0,13.0


A Pyomo's concrete model() was utilized to optimize the scheduling of operating rooms in a healthcare facility over a three-month period. By formulating the problem with decision variables, an objective function, and constraints, the aim was to maximize the number of scheduled procedures while considering various limitations.

The model was solved using the GLPK solver, yielding an optimal solution that maximized the number of scheduled procedures and adhered to the defined constraints. The results were presented in a concise table, showcasing the Encounter ID, operating room, day, hour, booked time, and end time for each scheduled procedure.

By leveraging Pyomo and the concrete model(), a scheduling team can be able to enhance resource utilization and increase the efficiency of the scheduling process for operating rooms. This approach offers versatility in handling different constraints and objectives, making it valuable for similar scheduling challenges in healthcare facilities.